In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

class CreditAnalyzer:
    def __init__(self, contamination=0.0153):
        self.scaler = StandardScaler()
        self.isolation_forest = IsolationForest(
            contamination=contamination,
            random_state=42,
            n_estimators=200
        )

        # 소비 패턴 관련 변수들
        self.consumption_features = [
            'TOT_USE_AM', 'CRDSL_USE_AM', 'CNF_USE_AM',
            'OFFEDU_AM_AVG', 'TRVLEC_AM', 'FSBZ_AM_AVG',
            'RESTRNT_AM_AVG', 'LEISURE_S_AM_AVG', 'LEISURE_P_AM_AVG'
        ]

        # 재무 상태 관련 변수들
        self.financial_features = [
            'ASSETS', 'DEBT', 'INCOME', 'DAN_RT', 'DEBT_RATIO',
            'UES_INCOME', 'CRDSL_DEP'
        ]

    def analyze_patterns(self, df):
        """소비 패턴 분석 - 수정된 버전"""
        patterns = []

        for feature in self.consumption_features + self.financial_features:
            # 원본 데이터(스케일링 전)에서 평균 계산
            normal_mean = df[~df['is_anomaly']][feature].mean()
            anomaly_mean = df[df['is_anomaly']][feature].mean()

            # 차이 계산 수정
            if normal_mean != 0:  # 0으로 나누기 방지
                diff_percent = ((anomaly_mean - normal_mean) / abs(normal_mean)) * 100
            else:
                diff_percent = 0 if anomaly_mean == 0 else np.inf

            patterns.append({
                'feature': feature,
                'normal_mean': normal_mean,
                'anomaly_mean': anomaly_mean,
                'difference_percent': diff_percent
            })

        patterns_df = pd.DataFrame(patterns)
        return patterns_df.sort_values('difference_percent', ascending=False)

    def generate_report(self, df, patterns):
        """분석 리포트 생성 - 수정된 버전"""
        print("\n=== 이상치 탐지 분석 보고서 ===")

        print("\n1. 기본 통계")
        print(f"전체 데이터 수: {len(df):,}")
        print(f"이상치 데이터 수: {df['is_anomaly'].sum():,}")
        print(f"이상치 비율: {(df['is_anomaly'].sum() / len(df)) * 100:.2f}%")

        print("\n2. 생애주기별 이상치 분포")
        life_stage_dist = pd.crosstab(df['LIFE_STAGE'], df['is_anomaly'], normalize='index') * 100
        print(life_stage_dist)

        print("\n3. 주요 특성 차이")
        print("\n[소비 패턴 차이 (상위 5개)]")
        consumption_patterns = patterns[patterns['feature'].isin(self.consumption_features)]
        print(consumption_patterns.head().round(2))

        print("\n[재무 지표 차이]")
        financial_patterns = patterns[patterns['feature'].isin(self.financial_features)]
        print(financial_patterns.round(2))

        # 추가: 이상치 그룹의 특성 상세 분석
        print("\n4. 이상치 그룹 특성 상세 분석")
        for feature in self.financial_features:
            normal_quantiles = df[~df['is_anomaly']][feature].quantile([0.25, 0.5, 0.75])
            anomaly_quantiles = df[df['is_anomaly']][feature].quantile([0.25, 0.5, 0.75])

            print(f"\n{feature} 분포:")
            print("정상 그룹 - 25/50/75 백분위수:", normal_quantiles.round(2).values)
            print("이상치 그룹 - 25/50/75 백분위수:", anomaly_quantiles.round(2).values)

def main():
    # 분석기 초기화
    analyzer = CreditAnalyzer()

    # 데이터 로드 (파일 경로를 실제 경로로 수정하세요)
    df = pd.read_csv('Derived_Variables.csv')

    # 이상치 탐지 수행
    anomaly_detector = IsolationForest(contamination=0.0153, random_state=42)
    df['is_anomaly'] = anomaly_detector.fit_predict(df[analyzer.consumption_features + analyzer.financial_features]) == -1

    # 패턴 분석
    patterns = analyzer.analyze_patterns(df)

    # 리포트 생성
    analyzer.generate_report(df, patterns)

if __name__ == "__main__":
    main()


=== 이상치 탐지 분석 보고서 ===

1. 기본 통계
전체 데이터 수: 90,001
이상치 데이터 수: 1,377
이상치 비율: 1.53%

2. 생애주기별 이상치 분포
is_anomaly      False     True 
LIFE_STAGE                     
CHILD_BABY  98.206713  1.793287
CHILD_TEEN  98.196177  1.803823
CHILD_UNI   98.198105  1.801895
GOLLIFE     98.733923  1.266077
NEW_JOB     99.457443  0.542557
NEW_WED     97.729932  2.270068
RETIR       97.851074  2.148926
SECLIFE     98.456309  1.543691
UNI         99.741468  0.258532

3. 주요 특성 차이

[소비 패턴 차이 (상위 5개)]
            feature  normal_mean  anomaly_mean  difference_percent
7  LEISURE_S_AM_AVG         5.13        130.54             2444.62
8  LEISURE_P_AM_AVG         1.86         26.23             1308.73
4         TRVLEC_AM        14.18        119.41              741.82
3     OFFEDU_AM_AVG        10.51         44.95              327.88
5       FSBZ_AM_AVG        64.19        234.38              265.12

[재무 지표 차이]
       feature  normal_mean  anomaly_mean  difference_percent
14  UES_INCOME         0.08          0.29

이 분석 결과는 매우 중요한 신용 위험 패턴을 보여주고 있습니다. 핵심적인 발견을 설명드리겠습니다:

# 생애주기별 리스크

- NEW_WED(신혼부부)가 2.27%로 가장 높은 이상치 비율
- RETIR(은퇴자)가 2.15%로 두 번째로 높음
- UNI(대학생)가 0.26%로 가장 낮음
> 신혼부부와 은퇴자 그룹에 대한 특별한 리스크 관리가 필요함을 시사


# 소비 패턴의 특이점

### 가장 큰 차이를 보이는 항목들:

- LEISURE_S_AM_AVG(여가 지출): 정상 5.13 vs 이상치 130.54 (2444% 증가)
- LEISURE_P_AM_AVG(여가 지출2): 정상 1.86 vs 이상치 26.23 (1308% 증가)
- TRVLEC_AM(여행 지출): 정상 14.18 vs 이상치 119.41 (741% 증가)
> 이상치 그룹이 여가/여행 등 비필수적 지출이 매우 높음




# 재무 지표의 위험 신호

### 주목할 만한 차이:

- UES_INCOME(소득 대비 사용금액): 정상 0.08 vs 이상치 0.29 (277% 증가)
- DAN_RT(연체율): 정상 0.09 vs 이상치 0.17 (83% 증가)
- CRDSL_DEP(신용카드 의존도): 정상 0.70 vs 이상치 0.60 (14% 감소)
> 이상치 그룹이 소득 대비 과도한 지출과 높은 연체율을 보임




# 상세 분포 분석

### DAN_RT(연체율)의 경우:

- 정상그룹 중앙값: 0.05
- 이상치그룹 중앙값: 0.15


# UES_INCOME(소득대비사용액)의 경우:

- 정상그룹 중앙값: 0.06
- 이상치그룹 중앙값: 0.28





# 핵심 시사점:

>이상치 그룹은 소득 수준에 비해 과도한 여가/여행 지출 패턴을 보임
>높은 연체율과 소득 대비 높은 지출 비율이 주요 위험 신호
>신혼부부와 은퇴자 그룹에 대한 특별한 관리가 필요

>이러한 결과는 신용평가 모델에 있어 소비 패턴, 특히 여가/여행 관련 지출과 소득 대비 사용금액 비율을 주요 지표로 활용할 필요가 있음을 시사합니다.